In [16]:
import numpy as np
import pandas as pd
from fast_histogram import histogram1d
from scipy import spatial
from shapely.geometry import Polygon, Point, MultiPoint
import matplotlib.pyplot as plt
from tqdm import tqdm as tqdm
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [266]:
def corr(features, density, r_min, r_max, dr):
    radius = features.r.mean()
    N = features.x.count()
    dists, orders, N = dists_and_orders(features, r_max * radius)
    r_values = np.arange(r_min, r_max, dr) * radius

    
    g, bins = np.histogram(dists, bins=r_values)
    g6, bins = np.histogram(dists, bins=r_values, weights=orders)
    bin_centres = bins[1:] - (bins[1] - bins[0]) / 2
    divisor = 2 * np.pi * r_values[:-1] * (bins[1] - bins[0]) * density * N

    g = g / divisor
    g6 = g6 / divisor
    return bin_centres, g, g6

def dists_and_orders(f, t=1000):
    idx = get_idx(f, t)
    dists = get_dists(f, idx)
    orders = get_orders(f, idx)
    return dists.ravel(), orders.ravel(), np.sum(idx)


def get_idx(f, t):
    return f.edge_distance.values > t
#     return f.x.values > 0


def get_dists(f, idx):
    x = f[['x', 'y']].values
    return spatial.distance.cdist(x[idx, :], x)


def get_orders(f, idx):
    orders = make_complex(f)
    order_grid = make_order_grid(orders, idx)
    return np.abs(order_grid)


def make_order_grid(orders, idx):
    orders = orders.reshape(-1, 1)
    return orders[idx] @ np.conj(orders).transpose()


def make_complex(f):
    return f['hexatic_order'].values


def flat_array(x):
    return np.concatenate([item.ravel() for item in x])

In [166]:
def add_edge_distance(data):
    points = data[['x', 'y']].values
    hull = spatial.ConvexHull(points)
    hull_points = points[hull.vertices, :]
    polygon = Polygon(hull_points)
    multi_point = MultiPoint(points)
    dists = [polygon.exterior.distance(p) for p in multi_point.geoms]
    data['edge_distance'] = dists
    return data, hull.volume

# Test one dataset

In [197]:
file = "/media/data/Data/BallBearing/HIPS/PhaseDiagramsNewPlate/2,25mm/85%/500.hdf5"
data = pd.read_hdf(file)
# data = add_edge_distance(data)


frame0 = data.loc[0].copy()
N = len(frame0)
frame0['r'] = frame0['size']/2
frame0, area = add_edge_distance(frame0)
r, g, g6 = corr(frame0, N/area, 1, 100, 0.1)
diameter = r[np.argmax(g)]

(7600,) (6197, 7600)
(47097200,) (47097200,) 6197


In [214]:
plt.subplot(1, 2, 1)
plt.loglog(r/diameter, g)
plt.xlim([0.4, max(r/diameter)])
plt.xlabel('r/d')
plt.ylabel('g(r)')
plt.subplot(1, 2, 2)
plt.loglog(r/diameter, g6/g)
plt.xlim([0.4, max(r/diameter)])
plt.xlabel('r/d')
plt.ylabel('$g_6(r)/g(r)$')

<ipython-input-214-25f91611a281>:7: RuntimeWarning: invalid value encountered in true_divide
  plt.loglog(r/diameter, g6/g)


Text(0, 0.5, '$g_6(r)/g(r)$')

In [215]:
files = filehandling.get_directory_filenames("/media/data/Data/BallBearing/HIPS/PhaseDiagramsNewPlate/2,25mm/85%/*.hdf5")

In [39]:
files = files[0]
data = pd.read_hdf(file)
data['r'] = data['size']/2
frame0 = data.loc[0].copy()

In [40]:
frame0, area = add_edge_distance(frame0)
frame0

,y,x,mass,size,ecc,signal,raw_mass,ep,hexatic_order,number_of_neighbors,user_rad,r,edge_distance
frame,,,,,,,,,,,,,
0,34.101056,621.652589,787.714417,1.844645,0.073402,50.296496,3279.0,0.004259,0.242043-0.071360j,4.0,6,0.922322,0.000000
0,35.296638,709.620933,730.289284,1.824454,0.106826,51.880638,2997.0,0.004660,0.656514+0.072891j,6.0,6,0.912227,0.865426
0,37.294147,732.122520,798.407373,1.880086,0.096788,47.128213,3408.0,0.004098,0.693728+0.555301j,4.0,6,0.940043,2.778472
0,36.950568,886.259516,801.179621,1.771834,0.014207,57.425133,3316.0,0.004211,0.323983+0.147130j,5.0,6,0.885917,1.856415
0,37.031296,925.499756,809.892400,1.804504,0.032419,56.237027,3267.0,0.004275,0.112866+0.127519j,5.0,6,0.902252,1.789872
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1956.960343,1341.948553,1108.503094,1.850137,0.023541,69.702231,4730.0,0.002951,0.413347-0.078628j,6.0,6,0.925069,0.000000
0,1956.433884,1409.812672,862.565109,1.861973,0.059557,55.048921,3962.0,0.003524,0.687178-0.408971j,6.0,6,0.930986,0.742090
0,1957.159343,1533.852567,964.346207,1.862744,0.150191,58.613240,4228.0,0.003302,0.484431+0.194901j,5.0,6,0.931372,0.410765


In [101]:
r_min, r_max, dr = 2, 100, 0.1
radius = frame0.r.mean()
N = frame0.x.count()
dists, orders, N = dists_and_orders(frame0, 0)#r_max * radius * 3)
r_values = np.arange(r_min, r_max, dr) * radius
density = len(frame0)/area

In [102]:
N, len(dists), frame0.x.count()

(7562, 57418266, 7593)

In [103]:
g, bins = np.histogram(dists, bins=r_values)
g6, bins = np.histogram(dists, bins=r_values, weights=orders)
bin_centres = bins[1:] - (bins[1] - bins[0]) / 2
divisor = 2 * np.pi * r_values[:-1] * (bins[1] - bins[0]) * density * len(
    dists)

In [118]:
plt.plot(r_values[:-1], divisor/len(dists))
plt.xlabel('r [pixels]')
plt.ylabel('$N_{expect}$')

Text(0, 0.5, '$N_{expect}$')

In [116]:
plt.plot(r_values[:-1], g/divisor)
plt.xlabel('r [pixels]')
plt.ylabel('g(r)')

Text(0, 0.5, 'g(r)')

In [270]:
files = filehandling.get_directory_filenames("/media/data/Data/BallBearing/HIPS/PhaseDiagramsNewPlate/2,15mm/85%/*.hdf5")
figure_data = {}
for i, file in tqdm(enumerate(files)):
    duty = int(file[-8:-5])
    data = pd.read_hdf(file)
    data['r'] = data['size']/2
    frame0 =data.loc[0].copy()
    frame0, area = add_edge_distance(frame0)
    density = len(frame0) / area
    r, g, g6 = corr(frame0, density, 1, 300, 0.1)
    if i == 0:
        diameter = r[np.argmax(g)]
    
    figure_data[duty] = (r, g, g6, diameter)
    


0it [00:00, ?it/s]
1it [00:05,  5.75s/it]
2it [00:11,  5.76s/it]
3it [00:17,  5.76s/it]
4it [00:22,  5.74s/it]
5it [00:28,  5.75s/it]
6it [00:34,  5.68s/it]
7it [00:40,  5.70s/it]
8it [00:45,  5.68s/it]
9it [00:51,  5.67s/it]
10it [00:56,  5.64s/it]
11it [01:02,  5.60s/it]
12it [01:08,  5.62s/it]
13it [01:13,  5.62s/it]
14it [01:19,  5.63s/it]
15it [01:24,  5.63s/it]
16it [01:30,  5.60s/it]
17it [01:36,  5.60s/it]
18it [01:41,  5.54s/it]
19it [01:47,  5.54s/it]
20it [01:52,  5.55s/it]
21it [01:58,  5.55s/it]
22it [02:03,  5.55s/it]
23it [02:09,  5.50s/it]
24it [02:14,  5.47s/it]
25it [02:19,  5.48s/it]
26it [02:25,  5.52s/it]
27it [02:31,  5.51s/it]
28it [02:36,  5.46s/it]
29it [02:41,  5.45s/it]
30it [02:47,  5.44s/it]
31it [02:52,  5.47s/it]
32it [02:58,  5.46s/it]
33it [03:03,  5.46s/it]
34it [03:09,  5.44s/it]
35it [03:14,  5.44s/it]
36it [03:19,  5.38s/it]
37it [03:25,  5.34s/it]
38it [03:30,  5.29s/it]
39it [03:35,  5.25s/it]
40it [03:40,  5.22s/it]
41it [03:45,  5.14s/it]
42it 

In [20]:
import pickle

In [271]:
with open(new_file, 'wb') as f:
    pickle.dump(figure_data, f)
    

# Plot

In [15]:
from matplotlib.widgets import Slider

In [18]:
direc1 = "/media/data/Data/BallBearing/HIPS/PhaseDiagramsNewPlate/2,15mm/85%"
new_file1 = direc1 + '/correlation_data.pkl'
direc2 = "/media/data/Data/BallBearing/HIPS/PhaseDiagramsNewPlate/2,25mm/85%"
new_file2 = direc2 + '/correlation_data.pkl'

In [21]:
with open(new_file1, 'rb') as f:
    figure_data_2_15 = pickle.load(f)

with open(new_file2, 'rb') as f:
    figure_data_2_25 = pickle.load(f)

In [24]:
fig, ax = plt.subplots(1, 2)
plt.subplots_adjust(bottom=0.25)
keys = list(figure_data_2_15.keys())
key = keys[0]
r, g, g6, diameter = figure_data_2_15[key]
r2, g2, g62, diameter2 = figure_data_2_25[key]
diameter = r[np.argmax(g)] / 2
plot1, = ax[0].semilogy(r*pix_2_mm, g-1)
plot1b, = ax[0].semilogy(r2*pix_2_mm, g2-1)
# ax[0].set_xlim([0.4, 5])
ax[0].set_xlabel('r [mm]')
ax[0].set_ylabel('g')
plot2, = ax[1].semilogy(r, g6/g)
plot2b, = ax[1].semilogy(r2, g62/g2)
# ax[1].set_xlim([0.4, 5])
ax[1].set_xlabel('r [mm]')
ax[1].set_ylabel('g6/g')


s_ax = plt.axes([0.25, 0.1, 0.65, 0.03])
slider = Slider(s_ax, 'Duty', min(keys), max(keys), valinit=key, valstep=1)

def update(val):
    duty = slider.val
    r, g, g6, diameter = figure_data_2_15[duty]
    r2, g2, g62, diameter2 = figure_data_2_25[duty]
    diameter = r[np.argmax(g)] / 2
    plot1.set_xdata(r*pix_2_mm)
    plot1.set_ydata(g)
    plot1b.set_xdata(r2*pix_2_mm)
    plot1b.set_ydata(g2)
    plot2.set_xdata(r*pix_2_mm)
    plot2.set_ydata(g6/g)
    plot2b.set_xdata(r2*pix_2_mm)
    plot2b.set_ydata(g62/g2)
slider.on_changed(update)



<ipython-input-24-1ec3e8f63c70>:13: RuntimeWarning: invalid value encountered in true_divide
  plot2, = ax[1].semilogy(r, g6/g)
<ipython-input-24-1ec3e8f63c70>:14: RuntimeWarning: invalid value encountered in true_divide
  plot2b, = ax[1].semilogy(r2, g62/g2)


0

<ipython-input-24-1ec3e8f63c70>:33: RuntimeWarning: invalid value encountered in true_divide
  plot2.set_ydata(g6/g)
<ipython-input-24-1ec3e8f63c70>:35: RuntimeWarning: invalid value encountered in true_divide
  plot2b.set_ydata(g62/g2)


# Test fake data

In [225]:
import trigrid

In [241]:
x, y = trigrid.grid(10, 100)
data = pd.DataFrame({'x': x, 'y': y, 'r': 5, 'hexatic_order': 1 + 0j})

In [242]:
data, area = add_edge_distance(data)

In [243]:
density = len(data)/area

In [250]:
r, g, g6 = corr(data, density, 2, 100, 0.1)

(40401,) (8543, 40401)
(345145743,) (345145743,) 8543


In [245]:
plt.loglog(r, g-1)

In [251]:
diameter = r[np.argmax(g)]/sqrt(3)

In [256]:
plt.loglog(r/diameter, g-1, '-o')
# for peak in peak_guesses:
#     plt.axvline(peak, c='r')

In [248]:
from math import sqrt
peak_guesses = [1, sqrt(3), 2, sqrt(7), 3]

# Get pixel to mm conversion

In [3]:
from labvision import images, video
file = "/media/data/Data/BallBearing/HIPS/PhaseDiagramsNewPlate/2,15mm/85%/451.mp4"
vid = video.ReadVideo(file)
frame = vid.read_frame()

In [4]:
images.display(frame)

[]

In [5]:
crop_result = images.crop_rectangle(frame)

In [10]:
bbox = crop_result.bbox

In [9]:
bbox

[[1324, 92], [2431, 2056]]

In [11]:
dx = bbox.xmax - bbox.xmin
dy = bbox.ymax - bbox.ymin

In [14]:
L = dy
pix_2_mm = 193.73 / L

In [25]:
L * pix_2_mm

193.73

In [27]:
dx, dy

(1107, 1964)

In [31]:
cropped = images.crop_and_mask(frame, crop_result.bbox, crop_result.mask)

In [ ]:
images.display(cropped)